In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
# Clone
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# Configure
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

In [7]:
# Make a copy of yolov3.cfg
!cp cfg/yolov3-tiny.cfg cfg/yolov3-tiny_training.cfg

In [8]:
# Change lines in yolov3.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov3-tiny_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3-tiny_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3-tiny_training.cfg
!sed -i '610 s@classes=80@classes=2@' cfg/yolov3-tiny_training.cfg
!sed -i '696 s@classes=80@classes=2@' cfg/yolov3-tiny_training.cfg
!sed -i '783 s@classes=80@classes=2@' cfg/yolov3-tiny_training.cfg
!sed -i '603 s@filters=255@filters=21@' cfg/yolov3-tiny_training.cfg
!sed -i '689 s@filters=255@filters=21@' cfg/yolov3-tiny_training.cfg
!sed -i '776 s@filters=255@filters=21@' cfg/yolov3-tiny_training.cfg

In [9]:
!echo -e "kering\nbasah" > data/obj.names
!echo -e 'classes= 2\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data

In [10]:
!cp cfg/yolov3-tiny_training.cfg /mydrive/yolov3/yolov3_testing.cfg
!cp data/obj.names /mydrive/yolov3/classes.txt

In [ ]:
!mkdir data/obj
!unzip /mydrive/yolov3/images.zip -d data/obj

In [ ]:
import glob
import os
import re
   
txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            # print(line)
            numbers = re.findall("[0-9.]+", line)
            # print(numbers)

            if numbers:
              if numbers[0] == '15':
                numbers[0] = 0
              if numbers[0] == '16':
                numbers[0] = 1
              if numbers[0] == '17':
                numbers[0] = 2
              
              # Define coordinates
              text = "{} {} {} {} {}".format(numbers[0], numbers[1], numbers[2], numbers[3], numbers[4])
              text_converted.append(text)
              # print(i, file_path)
              print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

In [ ]:
import glob
# Don't Forget The Image Extension
images_list = glob.glob("data/obj/*.JPG")
print(images_list)
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show